In [2]:
from bert_embedding import BertEmbedding
import pandas as pd
import numpy as np
import mxnet as mx
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
import pickle
import models
from gensim.models import Word2Vec

/home/mounika/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from gensim.models import KeyedVectors
w2vmodel = KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300-002.bin",binary=True)

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
def glove_features(data):
    x_hk_w2 = []
    for i in data:
        tokens = nlp(i)
        temp = []
        for token in tokens:
            try:
                temp.append(w2vmodel[str(token).lower()])
            except:
                continue
        temp = np.array(temp)
        if temp.size==0:
            temp = np.zeros([1,300])
        #print(temp.shape)
        x_hk_w2.append(np.average(temp,axis=0))
    x_hk_w2 = np.array(x_hk_w2)
    print(x_hk_w2.shape)
    return x_hk_w2

In [60]:
fp = open('./Video_Games/OnlyText.txt','r')
lines = fp.readlines()

In [61]:
text_data = []
for line in lines:
    text_data.append(line.strip('\n'))

In [62]:
print(len(text_data))

50000


In [63]:
x_features_hk_w2v = glove_features(text_data)

(50000, 300)


In [64]:
np.save('/home/mounika/Desktop/Word-Embeddings/dranziera_protocol/WORD2VEC_features/Video_Games_w2v.npy',x_features_hk_w2v)

In [327]:
x_features = np.load('./WORD2VEC_features/Clothing_Accessories_w2v.npy')

In [330]:
labels = pd.read_csv('./Clothing_Accessories/polarity.txt',header=None)

In [331]:
labels.shape

(50000, 1)

In [332]:
y = []
for i in labels[0]:
    if i=='positive':
        y.append(1)
    else:
        y.append(0)

In [333]:
y = np.array(y)

In [334]:
print(x_features.shape)
print(y.shape)

(50000, 300)
(50000,)


In [335]:
from sklearn.model_selection import train_test_split,cross_val_score
x_train,x_test,y_train,y_test = train_test_split(x_features,y,test_size=0.2,random_state=47)

In [336]:
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
import argparse

In [337]:

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", dest="model", type=str, default=None, help="gcfoest Net Model File")
    args = parser.parse_args()
    return args


def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 100
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 2
    ca_config["estimators"] = []
    ca_config["estimators"].append(
            {"n_folds": 5, "type": "LGBClassifier", "n_estimators": 10, "max_depth": 5,
             "objective": "binary", "silent": True, "nthread": 5, "learning_rate": 0.1} )
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"}),
    #ca_config["estimators"].append({"n_folds": 5, "type": "GradientBoostingClassifier"})
    config["cascade"] = ca_config
    return config

In [338]:
config = get_toy_config()
gc = GCForest(config)

In [339]:
gc.config

{'cascade': {'random_state': 0,
  'max_layers': 100,
  'early_stopping_rounds': 3,
  'n_classes': 2,
  'estimators': [{'n_folds': 5,
    'type': 'LGBClassifier',
    'n_estimators': 10,
    'max_depth': 5,
    'objective': 'binary',
    'silent': True,
    'nthread': 5,
    'learning_rate': 0.1},
   {'n_folds': 5,
    'type': 'RandomForestClassifier',
    'n_estimators': 10,
    'max_depth': None,
    'n_jobs': -1},
   {'n_folds': 5,
    'type': 'ExtraTreesClassifier',
    'n_estimators': 10,
    'max_depth': None,
    'n_jobs': -1},
   {'n_folds': 5, 'type': 'LogisticRegression'}]}}

In [340]:
X_train_enc = gc.fit_transform(x_train, y_train,X_test=x_test, y_test=y_test)
#y_pred = gc.predict(x_test)

[ 2019-03-26 21:41:18,556][cascade_classifier.fit_transform] X_groups_train.shape=[(40000, 300)],y_train.shape=(40000,),X_groups_test.shape=[(10000, 300)],y_test.shape=(10000,)
[ 2019-03-26 21:41:18,567][cascade_classifier.fit_transform] group_dims=[300]
[ 2019-03-26 21:41:18,568][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-03-26 21:41:18,569][cascade_classifier.fit_transform] group_ends=[300]
[ 2019-03-26 21:41:18,570][cascade_classifier.fit_transform] X_train.shape=(40000, 300),X_test.shape=(10000, 300)
[ 2019-03-26 21:41:18,589][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(40000, 300), X_cur_test.shape=(10000, 300)
[ 2019-03-26 21:41:19,411][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=85.25%
[ 2019-03-26 21:41:20,260][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=85.85%
[ 2019-03-26 21:41:21,065][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 -

[ 2019-03-26 21:41:51,171][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_0.predict)=95.19%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-26 21:41:52,445][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_1.predict)=95.16%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-26 21:41:53,790][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_2.predict)=94.99%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.


[ 2019-03-26 21:42:22,360][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_3.predict)=95.17%
[ 2019-03-26 21:42:23,290][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_4.predict)=95.34%
[ 2019-03-26 21:42:23,406][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_cv.predict)=95.27%
[ 2019-03-26 21:42:23,410][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.test.predict)=96.06%
[ 2019-03-26 21:42:23,983][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_0.predict)=95.31%
[ 2019-03-26 21:42:24,713][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_1.predict)=95.20%
[ 2019-03-26 21:42:25,473][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_2.predict)=95.50%
[ 2019-03-26 21:42:26,200][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_3.predict)=95.25%
[ 2019-03-

[ 2019-03-26 21:42:49,457][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=95.64%
[ 2019-03-26 21:42:49,457][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=96.08%
[ 2019-03-26 21:42:49,474][cascade_classifier.fit_transform] [layer=5] look_indexs=[0], X_cur_train.shape=(40000, 308), X_cur_test.shape=(10000, 308)
[ 2019-03-26 21:42:50,301][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_0.predict)=95.40%
[ 2019-03-26 21:42:51,108][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_1.predict)=95.58%
[ 2019-03-26 21:42:51,932][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_2.predict)=95.55%
[ 2019-03-26 21:42:52,741][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_3.predict)=95.62%
[ 2019-03-26 21:42:53,552][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_4.predict)=95.76%
[ 201

/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-26 21:43:21,078][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_1.predict)=95.48%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-26 21:43:22,043][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_2.predict)=95.46%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-26 21:43:23,062][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_3

In [341]:
y_pred = gc.predict(x_test)

[ 2019-03-26 21:43:24,050][cascade_classifier.transform] X_groups_test.shape=[(10000, 300)]
[ 2019-03-26 21:43:24,053][cascade_classifier.transform] group_dims=[300]
[ 2019-03-26 21:43:24,053][cascade_classifier.transform] X_test.shape=(10000, 300)
[ 2019-03-26 21:43:24,056][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(10000, 300)
[ 2019-03-26 21:43:25,287][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(10000, 308)
[ 2019-03-26 21:43:26,477][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(10000, 308)
[ 2019-03-26 21:43:27,633][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(10000, 308)


In [342]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      5012
           1       0.97      0.96      0.96      4988

   micro avg       0.96      0.96      0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000



In [343]:
import pickle
pickle.dump(gc,open('/home/mounika/Desktop/Word-Embeddings/dranziera_protocol/GC_w2v/gc_w2v_Clothing_Accessories','wb'))